# 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
import torch
from threading import Thread
import time
import sys 
# ----------------------------------------------------------------------------:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
#model_name = "meta-llama/Llama-3.2-3B-Instruct"
#model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# ----------------------------------------------------------------------------:

In [ ]:

if torch.cuda.is_available(): device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available(): device = "mps"
else: device = "cpu"
model = model.to(device)
print(f"DEBUG: Device set to: {device}")

In [ ]:
# https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]
chat_text = tokenizer.apply_chat_template(
    messages, 
    tokenize=True,
    add_generation_prompt=True
)
print(f"Chat prompt text ids:\n{chat_text}")
print(f"Chat prompt decoded:\n{tokenizer.decode(chat_text)}")
print("-"*40)
tokenized_chat = tokenizer.apply_chat_template(messages, 
                                               tokenize=True, 
                                               add_generation_prompt=True, 
                                               return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

In [ ]:
len(tokenized_chat[0]) 

In [ ]:
# (9125) --> system
# (882) --> user
# (128006) --> <|start_header_id|>
# (128007) --> <|end_header_id|> 
# (128009) --> <|eot_id|> 
# 
# so you will get a [<start_header_id> | <ID> | <end_header_id> ... <eot_id> ]
# This consites a block

for i, token in enumerate(tokenized_chat[0].tolist(), start=1) : 
    decoded = tokenizer.decode(token)
    if decoded == "\\n": decoded = "-newline-"
    print(f" ({token}) --> {decoded}")
print(f"Number of tokens in into prompt: {i}")

In [ ]:
chat_text = tokenizer.apply_chat_template(
    messages, 
    tokenize=False,
    add_generation_prompt=True
)
# Tokenize the formatted chat
inputs = tokenizer(chat_text, return_tensors="pt").to(device)
num_input_tokens = inputs.input_ids.shape[1]

print(inputs) 
print(inputs.input_ids)

In [ ]:
# Create streamer for token-by-token generation
streamer = TextIteratorStreamer(tokenizer, 
                                skip_prompt=True, 
                                skip_special_tokens=True)



# Start generation in a separate thread
generation_kwargs = {
    "input_ids": inputs.input_ids,
    "attention_mask": inputs.attention_mask,
    "max_new_tokens": 100,
    "temperature": 0.7,
    "streamer": streamer,
    "eos_token_id": None,
    "do_sample": True
}

thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()


prefill_start = time.time()
first_token = next(streamer)
prefill_end = time.time()

prefill_time = prefill_end - prefill_start



# Track generation time
generated_text = first_token
tokens_generated = 1

generation_start = time.time()
# Continue streaming tokens
for token in streamer:
    generated_text += token
    tokens_generated += 1

generation_end = time.time()
generation_time = generation_end - generation_start

# Print metrics
print(f"Input: {chat_text}")
print(f"Generated: {generated_text}")
print(f"Input tokens: {num_input_tokens}")
print(f"Output tokens: {tokens_generated}")
print(f"Prefill time: {prefill_time:.4f}s")
print(f"Generation time: {generation_time:.4f}s")
print(f"Generation tokens/second: {tokens_generated/generation_time:.2f}")

---
## LLM + KG 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
import torch
from threading import Thread
import time
import sys 
from mlhq.utils import load_jsonl, proceed, write_json 

/Users/msbabo/code/mlhq-kg-mac/mlhq/VENV/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/msbabo/code/mlhq-kg-mac/mlhq/VENV/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
#model_name = "meta-llama/Llama-3.2-3B-Instruct"
#model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
if torch.cuda.is_available(): device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available(): device = "mps"
else: device = "cpu"
model = model.to(device)
print(f"DEBUG: Device set to: {device}")

DEBUG: Device set to: mps


In [4]:
qa_paths = {
    "biomedical" : "/Users/msbabo/code/Graph-CoT/data/processed_data/biomedical/data.jsonl", 
    "amazon" : "/Users/msbabo/code/Graph-CoT/data/processed_data/amazon/data.jsonl", 
    "dblp" : "/Users/msbabo/code/Graph-CoT/data/processed_data/dblp/data.jsonl", 
    "goodreads": "/Users/msbabo/code/Graph-CoT/data/processed_data/goodreads/data.jsonl", 
    "legal": "/Users/msbabo/code/Graph-CoT/data/processed_data/legal/data.jsonl", 
    "maple-biology": "/Users/msbabo/code/Graph-CoT/data/processed_data/maple/Biology/data.jsonl",
    "maple-medicine": "/Users/msbabo/code/Graph-CoT/data/processed_data/maple/Medicine/data.jsonl",
    "maple-physics": "/Users/msbabo/code/Graph-CoT/data/processed_data/maple/Physics/data.jsonl",
    "maple-materials-science": "/Users/msbabo/code/Graph-CoT/data/processed_data/maple/Materials_Science/data.jsonl",
    "maple-chemistry": "/Users/msbabo/code/Graph-CoT/data/processed_data/maple/Chemistry/data.jsonl",
}
domains = [ 
    "amazon", 
    "biomedical", 
    "dblp", 
    "goodreads", 
    "legal", 
    "maple-biology", 
    "maple-physics", 
    "maple-chemistry", 
    "maple-medicine", 
    "maple-materials-science", 
]

In [5]:
system_prompt = """
You are an expert prompt classifier designed to categorize user prompts accurately into one of two distinct categories:

Internal Response: Prompts for which an LLM can confidently provide accurate, complete, and helpful responses solely using its internal knowledge, without needing external information, databases, APIs, or tooling.

External Response: Prompts for which an LLM requires external knowledge, updated information, APIs, databases, or external tooling to confidently deliver accurate, complete, and helpful responses.

Guidelines for Classification:

Classify prompts as "Internal Response" if the information needed to answer fully and confidently is within general world knowledge as of the training cutoff, and no specific, updated, or external information is required.

Classify prompts as "External Response" if answering correctly and confidently necessitates:

Real-time data, current events, or recent updates.

Information specific to locations, current weather, dates, times, recent publications, or market values.

External computation, databases, or web searches.

Provide only the classification label ("Internal Response" or "External Response") as your output.
"""
system_prompt_len = len(tokenizer(system_prompt)['input_ids'])
#print(f"Length of System-prompt: {system_prompt_len}")

#messages = [{"role": "system","content": f"{system_prompt}"}]
#sys_txt = tokenizer.apply_chat_template(
#    messages, 
#    tokenize=True,
#    add_generation_prompt=True,
#    return_tensors="pt" # w/ Tensor, w/o list
#).to(device)
#print(type(sys_txt), sys_txt)

#for i, token in enumerate(sys_txt[0].tolist(), start=1) : 
#    decoded = tokenizer.decode(token)
#    if decoded == "\\n": decoded = "-newline-"
#    print(f" ({token:6}) --> {decoded}")
#print(f"Number of tokens in into prompt: {i}")

In [6]:
prefix = "Incoming Prompt:\n"
prefix_len = len(tokenizer(prefix)['input_ids'])

#prx_txt = tokenizer.apply_chat_template(
#    messages, 
#    tokenize=True,
#    add_generation_prompt=True,
#    return_tensors="pt" # w/ Tensor, w/o list
#).to(device)

---

In [7]:
results = {}
results['model'] = model_name
results['model-config'] = model.config.__dict__
#results['task'] = task 
#results['forward-params'] = dict(pipe.__dict__["_forward_params"])
#results["generation-config"] = {} 
results["system-prompt"] = system_prompt
results["system-prompt-len"] = system_prompt_len
results["prefix"] = prefix
results["prefix-len"] = prefix_len
#print(f"System prompt length = {system_prompt_len}")
#print(f"Prefix length = {prefix_len}")

In [14]:
for domain in domains: 
    qa_data = load_jsonl(qa_paths[domain])

    results[domain] = {}
    results[domain]['high-level-metrics'] = {
        "total-questions" : 0,
        "total-internals" : 0, 
        "total-externals" : 0, 
        "total-latency-s" : 0.0, 
        "average-latency-s" : 0.0, 
    }
    results[domain]['per-question-results'] = [] 
    count = 10
    internals = 0 
    externals = 0 
    average_latency = 0.0
    total_latency = 0.0 
    for i, qdata in enumerate(qa_data, start=1): 
        if i > count: 
            i = i - 1 
            break 
        qid = qdata["qid"]
        question = qdata["question"]
        messages = [
            {
                "role": "system",
                "content": f"{system_prompt}"
            },
            {
                "role": "user", 
                "content": f"{prefix}{question}"
            }
        ]
        #inputs = tokenizer.apply_chat_template(
        #    messages, 
        #    tokenize=False,
        #    add_generation_prompt=False, # 
        #    #return_tensors="pt" # w/ Tensor, w/o list
        #)#.to(device)
        #inputs = tokenizer(chat_text, return_tensors="pt").to(device)
        print(f"{i}.) [{domain}] Question   --> {question}")
        #print(f"{i}.) [{domain}] Classified --> {classified}")
        chat_text = tokenizer.apply_chat_template(
            messages, 
            tokenize=False,
            add_generation_prompt=True
        )
        # Tokenize the formatted chat
        inputs = tokenizer(chat_text, return_tensors="pt").to(device)
        num_input_tokens = inputs.input_ids.shape[1]
        #print(inputs) 
        #print(inputs.input_ids)



        # Create streamer for token-by-token generation
        streamer = TextIteratorStreamer(tokenizer, 
                                skip_prompt=True, 
                                skip_special_tokens=True)

        generation_kwargs = {
            "input_ids": inputs.input_ids,
            "attention_mask": inputs.attention_mask,
            "max_new_tokens": 100,
            "temperature": 0.7,
            "streamer": streamer,
            #"eos_token_id": None,
            "do_sample": True
        }

        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        prefill_start = time.time()
        first_token = next(streamer)
        prefill_end = time.time()
        prefill_time = prefill_end - prefill_start

        # Track generation time
        generated_text = first_token 
        tokens_generated = 1

        generation_start = time.time()
        # Continue streaming tokens
        for token in streamer:
            generated_text += token
            tokens_generated += 1
        generation_end = time.time()
        generation_time = generation_end - generation_start

         # Print metrics
        #print(f"Input: {chat_text}")
        #print(f"Generated: {generated_text}")
        print(f"Input tokens: {num_input_tokens}")
        print(f"Output tokens: {tokens_generated}")
        print(f"Prefill time: {prefill_time:.4f}s")
        print(f"Generation time: {generation_time:.4f}s")
        print(f"Generation tokens/second: {tokens_generated/generation_time:.2f}")
        
        #average_latency = 0.0
        total_latency += (prefill_time + generation_time) 
        
        if "Internal Response" in generated_text: 
            classified = "Internal Response"
            internals += 1 
        elif "External Response" in generated_text: 
            classified = "External Response"
            externals += 1
        else: 
            raise RuntimeError("bad llm response")
        # SimpleAgent("sys + prefix + prompt").SimpleAgent("sys2 + prefix2")
        results[domain]['per-question-results'].append(
            {'qid':qid, 
             'question':question, 
             "classified":classified,
             "num-input-tokens":num_input_tokens, 
             "num-output-tokens": tokens_generated, 
             "ttft": prefill_time, 
             "total-latency": generation_time, 
             "tps": tokens_generated/generation_time, 
            }
        )
        #proceed() 
    results[domain]['high-level-metrics']["total-questions"] = i 
    results[domain]['high-level-metrics']["total-internals"] = internals
    results[domain]['high-level-metrics']["total-externals"] = externals
    results[domain]['high-level-metrics']["total-latency-s"] = total_latency
    results[domain]['high-level-metrics']["average-latency-s"] = total_latency / i 
    



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1.) [amazon] Question   --> Could you specify the brand of Dolica DC-BP511 1400mAh Canon Battery?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input tokens: 271
Output tokens: 7
Prefill time: 0.1485s
Generation time: 0.1004s
Generation tokens/second: 69.75
2.) [amazon] Question   --> Could you specify the brand of [Aftermarket Product] Black PU Leather Wireless Bluetooth Keyboard Flip Case Cover For Samsung Galaxy S3 i9300 Sprint L710 att i747 Verizon i535 T-Mobile T999 New?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Input tokens: 279
Output tokens: 43
Prefill time: 0.0698s
Generation time: 0.9067s
Generation tokens/second: 47.43
8.) [maple-materials-science] Question   --> Can you tell me the authors of the paper titled "tensile properties of high strength polyacrylonitrile pan based and high modulus pitch based hybrid carbon fibers reinforced epoxy matrix composite?"


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input tokens: 291
Output tokens: 40
Prefill time: 0.0754s
Generation time: 0.7977s
Generation tokens/second: 50.14
9.) [maple-materials-science] Question   --> Can you tell me the authors of the paper titled "hydration of mgo 100 surface promoted at 011 steps?"


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input tokens: 276
Output tokens: 50
Prefill time: 0.0737s
Generation time: 1.0447s
Generation tokens/second: 47.86
10.) [maple-materials-science] Question   --> Who wrote the paper "microstructure and growth model for rice hull derived sic whiskers?"
Input tokens: 270
Output tokens: 6
Prefill time: 0.0724s
Generation time: 0.0774s
Generation tokens/second: 77.56


In [15]:
from mlhq.utils import get_datetime_str
revised_model_name = model_name.replace('/','--')
write_json(results, f"./baseline-{revised_model_name}-{get_datetime_str()}.json")

In [11]:
num_input_tokens = []
num_output_tokens = []
ttfts =[] 
total_latencies = [] 
tpss = [] 

for k,v in results.items(): 
    #print(k)
    if k in domains:
        for q in v: 
            num_input_tokens.append(q['num-input-tokens'])
            num_output_tokens.append(q['num-output-tokens'])
            ttfts.append(q['ttft'])
            total_latencies.append(q['total-latency'])
            tpss.append(q['tps'])
        

TypeError: string indices must be integers

In [ ]:
tpss

In [ ]:
# Plots tpss (y) as a function of output tokens lenght(x) and input tokens
import matplotlib.pyplot as plt

# Assuming num_output_tokens and tpss are already populated from your loop above

plt.figure(figsize=(10,6))

# Scatter plot
plt.scatter(num_output_tokens, tpss, marker='o')

# Adding titles and labels
plt.title(f'TPS vs. Number of Output Tokens [{model_name}]', fontsize=14)
plt.xlabel('Number of Output Tokens', fontsize=12)
plt.ylabel('TPS', fontsize=12)

# Optional: adding grid for better readability
plt.grid(True, linestyle='--', alpha=0.6)

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline

# Convert to numpy arrays
x = np.array(num_output_tokens)
y = np.array(tpss)

# Average TPS values for each unique output token count
unique_x = np.unique(x)
average_y = np.array([y[x == val].mean() for val in unique_x])

# Spline interpolation
x_smooth = np.linspace(unique_x.min(), unique_x.max(), 300)
spl = make_interp_spline(unique_x, average_y, k=3)
y_smooth = spl(x_smooth)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(x, y, marker='o', label='Original Data')
plt.plot(x_smooth, y_smooth, color='red', linewidth=2, label='Spline Curve (Averaged)')

# Labels and Grid
plt.title(f'TPS vs. Number of Output Tokens [{model_name}]', fontsize=14)
plt.xlabel('Number of Output Tokens', fontsize=12)
plt.ylabel('Tokens/Second', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()

plt.show()
